In [3]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
%matplotlib inline
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph
from sklearn.manifold import TSNE
# from fastText import tokenize, load_model
import gensim
import glob
import string
import pickle

In [7]:
df1 = pd.read_csv("grantnav-20180526072920.csv",encoding='utf-8')
df1 = df1[['Title', 'Description']]
df1 = df1.dropna()

/home/karmanya/anaconda3/envs/jupyter/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,10,13,19,20,21,23,24,25,31,34,35,36,37,38,39,40,41,42,43,44,45,46,47,49,50,51,53,54,55,57,58,59,61,62,63,65,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
descs = set(df1[~df1['Description'].isin(['Placeholder', 'None', 'donation','Ynot!', 'A', 'a', '-', 'none', '004649'])].Description.values)
# Removing some garbage values

In [24]:
titles = set(df1[~df1['Title'].isin(['.','C','-','x',])].Title.values)

Descriptions should be somewhat cleaned now in terms of entries and values. 

In [10]:
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    normalized = " ".join(lemma.lemmatize(word,'v') for word in stop_free.split())
    x = normalized.split()
    y = [s for s in x if len(s) > 2]
    return y

In [26]:
# import nltk
# nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/karmanya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
doc_clean = [clean(doc) for doc in descs]

In [30]:
from gensim import corpora
# Creating term dictionary of corpus, where each unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)
# Filter terms which occurs in less than 4 articles & more than 40% of the articles 
dictionary.filter_extremes(no_below=4, no_above=0.4)
filter_worlds = ['grant', 'award', 'fund', 'help', 'use', 'group', 'charity', 'programme', 'meet', 'make', 'towards', 'provide', 'aim', 'develop']
stop_ids = [dictionary.token2id[stopword] for stopword in filter_worlds if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)

In [31]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [32]:
# from gensim.models.ldamodel import LdaModel as Lda
from gensim import models
# Creating the object for LDA model using gensim library & Training LDA model on the document term matrix.
lda = models.LdaMulticore(doc_term_matrix, id2word=dictionary, num_topics=50, workers=3)
# ldamodel = Lda(doc_term_matrix, num_topics=50, id2word = dictionary, passes=50, iterations=500)

In [33]:
for i,topic in enumerate(lda.print_topics(num_topics=50, num_words=20)):
   words = topic[1].split("+")
   print((i,words),"\n")

(0, ['0.038*"anniversary" ', ' 0.035*"end" ', ' 0.026*"60th" ', ' 0.025*"one" ', ' 0.023*"commemorate" ', ' 0.022*"hold" ', ' 0.020*"war" ', ' 0.018*"world" ', ' 0.017*"royal" ', ' 0.015*"ww2" ', ' 0.015*"veterans" ', ' 0.015*"branch" ', ' 0.014*"exhibition" ', ' 0.013*"event" ', ' 0.013*"veteran," ', ' 0.013*"party" ', ' 0.013*"street" ', ' 0.013*"association" ', ' 0.013*"second" ', ' 0.012*"war."']) 

(1, ['0.021*"hear" ', ' 0.020*"system" ', ' 0.015*"new" ', ' 0.015*"people" ', ' 0.015*"increase" ', ' 0.014*"train" ', ' 0.012*"purchase" ', ' 0.010*"enable" ', ' 0.009*"deaf" ', ' 0.009*"support" ', ' 0.009*"improve" ', ' 0.009*"membership" ', ' 0.007*"system." ', ' 0.007*"justice" ', ' 0.007*"quality" ', ' 0.007*"portable" ', ' 0.006*"resources," ', ' 0.006*"loop" ', ' 0.006*"ladies" ', ' 0.006*"club"']) 

(2, ['0.064*"play" ', ' 0.050*"school" ', ' 0.039*"outdoor" ', ' 0.036*"children" ', ' 0.030*"area" ', ' 0.019*"create" ', ' 0.019*"equipment" ', ' 0.017*"garden" ', ' 0.016*"impro

In [34]:
# ldafile = open('multicore simple lda','wb')
# pickle.dump(lda,ldafile)
# ldafile.close()

In [4]:
ldafile = open('multicore simple lda','rb')
lda = pickle.load(ldafile)
ldafile.close()

In [12]:
def cluster_similar_documents(corpus):
    clean_docs = [clean(doc) for doc in corpus] #descs
    test_term = [lda.id2word.doc2bow(doc) for doc in clean_docs]
    doc_topics = lda.get_document_topics(test_term, minimum_probability=0.20)    
    return zip(doc_topics,corpus)

In [13]:
clustered = list(cluster_similar_documents(descs))

In [67]:
clustered[1722]

([(4, 0.24390677)],
 'This group will use the grant to purchase a new vehicle which will then be converted into a rescue ambulance for the Scottish region of the organisation. The ambulance, based in Grangemouth, will be custom built in order to transport a rescue team to an incident then move a trauma casualty to hospital.')

In [68]:
maps = {
49 : 'cultural activities',
48 : 'medical/health',
47 : 'sports',
46 : 'drug rehab/health',
45 : 'community activities',
44 : 'school activities/sports',
43 : 'counseling and support',
42 : 'commemorative events',
41: ' equipment/infrastructure',
40: 'cultural activities for youth',
39: 'wildlife and biology',
38: 'internal costs/salaries/internal',
37: 'mostly locations',
36: 'farming and agriculture',
35: 'health and awareness',
34: 'disabled youth activities',
33: 'infrastructure and equipment purchase',
32: 'salaries and costs',
31: 'construction and buildings',
30: 'equipment purchase',
29: 'science',
28: 'biology and medicine',
27: 'history and arts',
26: 'asylum seekers and refugees',
25: 'science',
24: 'cultural activites, film for disabilities',
23: 'skills development and employment',
21: 'science/medical research',
22: 'science/medical research',
20: 'causes in scotland',
19: 'land and wildlife',
18: 'medical/health/research',
17: 'infrastructure/plants and wildlife',
16: 'mental health',
15: 'local community events',
14: 'health children and young parents',
13: 'sports and cultural facilities',
12: 'summer school programmes',
11: 'community exercise and fitness/health programs',
10: 'local recreational activities',
9: 'local recreational activities/ cultural activities',
8: 'literacy and language support/education',
7: 'education and training and information services',
6: 'digital equipment',
5: 'awareness about food and health',
4: 'health/medical equipment purchase',
3: 'abused women and community support',
2: 'school/youth community sports/activities',
1: 'disabled people and minority support',
0: 'commerative events/veteran events'}

In [15]:
df2 = pd.DataFrame(clustered)
df2.to_csv("clustered.csv")

In [66]:
###### df2[0] = df2[0].astype(str)
df2[df2[0].str.contains('\(4,')]

,0,1
563,"[(4, 0.23569047), (17, 0.2518141), (30, 0.2080...",To contribute towards the equipping and mainte...
1722,"[(4, 0.24390677)]",This group will use the grant to purchase a ne...
2040,"[(4, 0.24566717), (7, 0.25401354), (32, 0.2160...",Toward the cost of work to diversify the incom...
2687,"[(4, 0.24210344), (31, 0.36671007)]",The Horizon Group's project is to establish a ...
2720,"[(4, 0.5006721), (34, 0.25932792)]",The Specialist Team Vehicle
3309,"[(4, 0.25535154), (15, 0.50464845)]",Cumnock Christmas Festival
3895,"[(4, 0.27877912), (47, 0.6706946)]",Funding under Sport England's CSP Funding 2011...
4208,"[(4, 0.20868544), (33, 0.39768374), (35, 0.205...",Towards training of volunteers as sighted guides
4545,"[(4, 0.20076282), (31, 0.32781005)]",We are raising money to help provide a safe tr...
5372,"[(4, 0.41667253), (29, 0.34332746)]",Renewal of bursaries support
